# Comprehensive Benchmarking Pipeline with Excel Export

This notebook provides a complete workflow for:
1. Running comprehensive benchmarks across multiple models and datasets
2. Generating standardized summary files
3. Creating formatted Excel reports with performance rankings

**Models included:**
- **AbLang Family**: AbLangPDB, AbLangRBD, AbLangPre, AbLang2, AbLang-Heavy (cosine similarity)
- **Other Protein LMs**: AntiBERTy, BALM, ESM-2, IgBERT, Parapred (cosine similarity)
- **Structure-based**: ABodyBuilder2 (Dynamic Time Warping over CDRs $e^{-(DTW/2)^2}$) 
- **Sequence-based**: SEQID (sequence identity), CDRH3ID (CDRH3 identity)

**Datasets:**
- SAbDab (structural antibody database)
- DMS (deep mutational scanning)

**Features:**
- Works with ALL available parquet files in the benchmarking directory
- Optional embedding re-calculation toggle (disabled by default)
- Optional threshold reuse for faster re-runs
- Comprehensive error handling and logging
- Supports up to 12+ different models across both datasets

## Configuration Flags

In [1]:
# =============================================================================
# CONFIGURATION FLAGS - MODIFY THESE AS NEEDED
# =============================================================================

# Set to False to use existing parquet files (faster), True to re-calculate embeddings
# NOTE: This notebook now works with ALL available parquet files in the directory
RECALCULATE_EMBEDDINGS = False

# Set to False to skip benchmark recalculation if summary files exist (faster), True to always recalculate
RECALCULATE_SUMMARYMETRICS = False

# Model paths - update these paths as needed
MODEL_PATHS = {
    "AbLangPDB": "../../../huggingface/AbLangPDB1/ablangpdb_model.safetensors",
    "AbLangRBD": "../../../huggingface/AbLangRBD1/model.safetensors"
}

# Batch size for embedding generation
BATCH_SIZE = 256

# Output configuration
OUTPUT_FOLDER = "output_csvs"
EXCEL_FILENAME = "comprehensive_benchmarking_results.xlsx"

print(f"Configuration:")
print(f"  • Recalculate embeddings: {RECALCULATE_EMBEDDINGS}")
print(f"  • Recalculate summary metrics: {RECALCULATE_SUMMARYMETRICS}")
print(f"  • Output folder: {OUTPUT_FOLDER}")
print(f"  • Excel filename: {EXCEL_FILENAME}")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"\n📝 Note: This notebook now supports ALL available parquet files:")
print(f"  • Models: AbLangPDB, AbLangRBD, AbLangPre, AbLang2, AbLang-Heavy,")
print(f"           AntiBERTy, BALM, ESM-2, IgBERT, Parapred, SEQID, CDRH3ID,")
print(f"           ABodyBuilder2_DTW_CDRs")
print(f"  • Datasets: SAbDab and DMS")
print(f"  • Only existing parquet files will be processed")
print(f"  • Summary metrics will be {'recalculated' if RECALCULATE_SUMMARYMETRICS else 'reused if available'}")

Configuration:
  • Recalculate embeddings: False
  • Recalculate summary metrics: False
  • Output folder: output_csvs
  • Excel filename: comprehensive_benchmarking_results.xlsx
  • Batch size: 256

📝 Note: This notebook now supports ALL available parquet files:
  • Models: AbLangPDB, AbLangRBD, AbLangPre, AbLang2, AbLang-Heavy,
           AntiBERTy, BALM, ESM-2, IgBERT, Parapred, SEQID, CDRH3ID,
           ABodyBuilder2_DTW_CDRs
  • Datasets: SAbDab and DMS
  • Only existing parquet files will be processed
  • Summary metrics will be reused if available


## Setup and Testing

In [2]:
import pandas as pd
import torch
import numpy as np
import os
import sys
from pathlib import Path
import warnings
import typing as T
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

# Add parent directory to path for imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import local modules
import calculate_metrics
import calculate_metrics_dms
import models
from excel_generator import generate_results_excel, print_summary_stats
from ablangpaired_model import AbLangPairedConfig, AbLangPaired

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure output directory exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

Using device: cuda


## Helper Functions

In [3]:
def check_file_exists(filepath, description=""):
    """Check if a file exists and return status."""
    exists = os.path.exists(filepath)
    status = "✅" if exists else "❌"
    print(f"  {status} {description}: {filepath}")
    return exists


def embed_with_ablangpaired(input_path: str, output_path: str, model_path: str, model_name: str):
    """Generate embeddings using AbLangPaired models.
    
        Args:
            model_name: str. If "ablangpre" then the model architecture will no longer have the mixer layer.
    """
    print(f"\n🔄 Generating {model_name} embeddings...")
    
    # Load data
    df = pd.read_parquet(input_path)
    if "EMBEDDING" in df.columns:
        df = df.drop(columns=["EMBEDDING"])
    
    # Setup model
    model_config = AbLangPairedConfig(checkpoint_filename=model_path)
    is_ablangpre = model_name == "ablangpre"
    model = AbLangPaired(model_config, device=device, use_pretrained=is_ablangpre)
    
    # Tokenize and embed using enhanced methods
    tokenized_dataloader = models.tokenize_data(df, model_config, batch_size=BATCH_SIZE)
    all_embeds = models.embed_dataloader(tokenized_dataloader, model, device)
    
    # Save
    df['EMBEDDING'] = list(all_embeds.cpu().numpy())
    df.to_parquet(output_path)
    
    print(f"✅ {model_name} embeddings saved to {output_path}")
    return df


def get_summary_file_paths(model_name: str, dataset_type: str, output_folder: str):
    """Get expected summary file paths for a model/dataset combination."""
    if dataset_type == "sabdab":
        # SAbDab has both epitope and antigen summary files
        epitope_file = os.path.join(output_folder, f"{model_name}_sabdab_ep_summarymetrics.txt")
        antigen_file = os.path.join(output_folder, f"{model_name}_sabdab_ag_summarymetrics.txt")
        return [epitope_file, antigen_file]
    elif dataset_type == "dms":
        # DMS has only one summary file
        dms_file = os.path.join(output_folder, f"{model_name}_dms_summarymetrics.txt")
        return [dms_file]
    else:
        return []


def read_and_display_summary_results(summary_files: list, model_name: str, dataset_type: str):
    """Read and display results from existing summary files."""
    print(f"\n📊 Loading existing results for {model_name} on {dataset_type}:")
    
    for summary_file in summary_files:
        if os.path.exists(summary_file):
            try:
                with open(summary_file, 'r') as f:
                    content = f.read().strip()
                
                # Extract the task type from filename
                if "sabdab_ep" in summary_file:
                    task_type = "Epitope-level performance"
                elif "sabdab_ag" in summary_file:
                    task_type = "Antigen-level performance"
                else:
                    task_type = "Performance"
                
                print(f"\n--- {task_type} ---")
                print(content)
                
            except Exception as e:
                print(f"❌ Error reading {summary_file}: {str(e)}")
        else:
            print(f"⚠️ Summary file not found: {summary_file}")


print("Helper functions loaded successfully.")

Helper functions loaded successfully.


## Data Preparation and Embedding Generation

### Check Required Base Files

In [4]:
# Check that base dataset files exist
print("📋 Checking base dataset files...")

base_files = {
    "SAbDab base dataset": "ablangpdb_renameddatasets.parquet",
    "DMS base dataset": "ablangrbd_renameddatasets.parquet",
    "SAbDab validation labels": "ablangpdb_train_val_label_mat.pt",
    "SAbDab test labels": "ablangpdb_train_test_label_mat.pt",
    "DMS validation labels": "dms_train_val_label_mat.pt",
    "DMS test labels": "dms_train_test_label_mat.pt"
}

missing_base_files = []
for desc, filepath in base_files.items():
    if not check_file_exists(filepath, desc):
        missing_base_files.append(filepath)

if missing_base_files:
    raise FileNotFoundError(f"Missing required base files: {missing_base_files}")

print("\n✅ All base files found!")

📋 Checking base dataset files...
  ✅ SAbDab base dataset: ablangpdb_renameddatasets.parquet
  ✅ DMS base dataset: ablangrbd_renameddatasets.parquet
  ✅ SAbDab validation labels: ablangpdb_train_val_label_mat.pt
  ✅ SAbDab test labels: ablangpdb_train_test_label_mat.pt
  ✅ DMS validation labels: dms_train_val_label_mat.pt
  ✅ DMS test labels: dms_train_test_label_mat.pt

✅ All base files found!


### Generate Embeddings (if needed)

In [5]:
# Define all embedding files that should exist
embedding_files = {
    # SAbDab dataset embeddings
    "sabdab_embeddedby_ablangrbd.parquet": ("ablangpdb_renameddatasets.parquet", MODEL_PATHS.get("AbLangRBD"), "AbLangRBD"),
    "sabdab_embeddedby_ablangpre.parquet": ("ablangpdb_renameddatasets.parquet", None, "AbLangPre"),
    "sabdab_embeddedby_ablang2.parquet": ("ablangpdb_renameddatasets.parquet", None, "AbLang2"),
    "sabdab_embeddedby_ablang-heavy.parquet": ("ablangpdb_renameddatasets.parquet", None, "AbLang-Heavy"),
    "sabdab_embeddedby_antiberty.parquet": ("ablangpdb_renameddatasets.parquet", None, "AntiBERTy"),
    "sabdab_embeddedby_balm.parquet": ("ablangpdb_renameddatasets.parquet", None, "BALM"),
    "sabdab_embeddedby_esm-2.parquet": ("ablangpdb_renameddatasets.parquet", None, "ESM-2"),
    "sabdab_embeddedby_igbert.parquet": ("ablangpdb_renameddatasets.parquet", None, "IgBERT"),
    "sabdab_embeddedby_parapred.parquet": ("ablangpdb_renameddatasets.parquet", None, "Parapred"),

    # DMS dataset embeddings
    "dms_embeddedby_ablangpdb.parquet": ("ablangrbd_renameddatasets.parquet", MODEL_PATHS.get("AbLangPDB"), "AbLangPDB"),
    "dms_embeddedby_ablangpre.parquet": ("ablangrbd_renameddatasets.parquet", None, "AbLangPre"),
    "dms_embeddedby_ablang2.parquet": ("ablangrbd_renameddatasets.parquet", None, "AbLang2"),
    "dms_embeddedby_ablang-heavy.parquet": ("ablangrbd_renameddatasets.parquet", None, "AbLang-Heavy"),
    "dms_embeddedby_antiberty.parquet": ("ablangrbd_renameddatasets.parquet", None, "AntiBERTy"),
    "dms_embeddedby_balm.parquet": ("ablangrbd_renameddatasets.parquet", None, "BALM"),
    "dms_embeddedby_esm-2.parquet": ("ablangrbd_renameddatasets.parquet", None, "ESM-2"),
    "dms_embeddedby_igbert.parquet": ("ablangrbd_renameddatasets.parquet", None, "IgBERT"),
    "dms_embeddedby_parapred.parquet": ("ablangrbd_renameddatasets.parquet", None, "Parapred")
}

print(f"\n{'='*60}")
print("EMBEDDING FILE STATUS CHECK")
print(f"{'='*60}")

print(" Checking for existing embedding files (no regeneration)...")

existing_files = []
missing_files = []

for output_file, (input_file, model_path, model_name) in embedding_files.items():
    if os.path.exists(output_file):
        print(f"✅ Found: {output_file}")
        existing_files.append(output_file)
    else:
        print(f"❌ Missing: {output_file}")
        missing_files.append(output_file)

print(f"\n📊 Embedding Files Summary:")
print(f"  • Existing files: {len(existing_files)}/{len(embedding_files)}")
print(f"  • Missing files: {len(missing_files)}")

if missing_files:
    print(f"\n⚠️ Missing embedding files:")
    for file in missing_files:
        print(f"  - {file}")
    print("\nNote: Configurations using missing files will be skipped automatically.")

print(f"\n✅ Embedding file check complete!")


EMBEDDING FILE STATUS CHECK
 Checking for existing embedding files (no regeneration)...
✅ Found: sabdab_embeddedby_ablangrbd.parquet
✅ Found: sabdab_embeddedby_ablangpre.parquet
✅ Found: sabdab_embeddedby_ablang2.parquet
✅ Found: sabdab_embeddedby_ablang-heavy.parquet
✅ Found: sabdab_embeddedby_antiberty.parquet
✅ Found: sabdab_embeddedby_balm.parquet
✅ Found: sabdab_embeddedby_esm-2.parquet
✅ Found: sabdab_embeddedby_igbert.parquet
✅ Found: sabdab_embeddedby_parapred.parquet
✅ Found: dms_embeddedby_ablangpdb.parquet
✅ Found: dms_embeddedby_ablangpre.parquet
✅ Found: dms_embeddedby_ablang2.parquet
✅ Found: dms_embeddedby_ablang-heavy.parquet
✅ Found: dms_embeddedby_antiberty.parquet
✅ Found: dms_embeddedby_balm.parquet
✅ Found: dms_embeddedby_esm-2.parquet
✅ Found: dms_embeddedby_igbert.parquet
✅ Found: dms_embeddedby_parapred.parquet

📊 Embedding Files Summary:
  • Existing files: 18/18
  • Missing files: 0

✅ Embedding file check complete!


### Verify All Required Files

In [ ]:
# Check that all required files now exist
print("\n📋 Verifying all required files...")

all_required_files = {
    # Base datasets
    "SAbDab base (AbLangPDB embeddings)": "ablangpdb_renameddatasets.parquet",
    "DMS base": "ablangrbd_renameddatasets.parquet",
    
    # SAbDab embedding files
    "SAbDab + AbLangRBD": "sabdab_embeddedby_ablangrbd.parquet",
    "SAbDab + AbLangPre": "sabdab_embeddedby_ablangpre.parquet",
    "SAbDab + AbLang2": "sabdab_embeddedby_ablang2.parquet",
    "SAbDab + AbLang-Heavy": "sabdab_embeddedby_ablang-heavy.parquet",
    "SAbDab + AntiBERTy": "sabdab_embeddedby_antiberty.parquet",
    "SAbDab + BALM": "sabdab_embeddedby_balm.parquet",
    "SAbDab + ESM-2": "sabdab_embeddedby_esm-2.parquet",
    "SAbDab + IgBERT": "sabdab_embeddedby_igbert.parquet",
    "SAbDab + Parapred": "sabdab_embeddedby_parapred.parquet",
    
    # DMS embedding files
    "DMS + AbLangPDB": "dms_embeddedby_ablangpdb.parquet",
    "DMS + AbLangPre": "dms_embeddedby_ablangpre.parquet",
    "DMS + AbLang2": "dms_embeddedby_ablang2.parquet",
    "DMS + AbLang-Heavy": "dms_embeddedby_ablang-heavy.parquet",
    "DMS + AntiBERTy": "dms_embeddedby_antiberty.parquet",
    "DMS + BALM": "dms_embeddedby_balm.parquet",
    "DMS + ESM-2": "dms_embeddedby_esm-2.parquet",
    "DMS + IgBERT": "dms_embeddedby_igbert.parquet",
    "DMS + Parapred": "dms_embeddedby_parapred.parquet",

    # ABodyBuilder2 DTW Pre-computed results
    "SAbDab + ABodyBuilder2 (for thresholding)": "sabdab_pairwise_cdr_sims_train_vs_val.npy",
    "SAbDab + ABodyBuilder2": "sabdab_pairwise_cdr_sims_train_vs_test.npy",
    "DMS + ABodyBuilder2 (for thresholding)": "dms_pairwise_cdr_sims_train_vs_val.npy",
    "DMS + ABodyBuilder2": "dms_pairwise_cdr_sims_train_vs_test.npy",
    
    # Label matrices
    "SAbDab validation labels": "ablangpdb_train_val_label_mat.pt",
    "SAbDab test labels": "ablangpdb_train_test_label_mat.pt",
    "DMS validation labels": "dms_train_val_label_mat.pt",
    "DMS test labels": "dms_train_test_label_mat.pt"
}

missing_files = []
for desc, filepath in all_required_files.items():
    if not check_file_exists(filepath, desc):
        missing_files.append(filepath)

if missing_files:
    print(f"\n⚠️ Warning: {len(missing_files)} files are missing:")
    for file in missing_files:
        print(f"  - {file}")
    print("\nProceeding with available files only.")
else:
    print("\n✅ All required files are available!")


📋 Verifying all required files...
  ✅ SAbDab base (AbLangPDB embeddings): ablangpdb_renameddatasets.parquet
  ✅ DMS base: ablangrbd_renameddatasets.parquet
  ✅ SAbDab + AbLangRBD: sabdab_embeddedby_ablangrbd.parquet
  ✅ SAbDab + AbLangPre: sabdab_embeddedby_ablangpre.parquet
  ✅ SAbDab + AbLang2: sabdab_embeddedby_ablang2.parquet
  ✅ SAbDab + AbLang-Heavy: sabdab_embeddedby_ablang-heavy.parquet
  ✅ SAbDab + AntiBERTy: sabdab_embeddedby_antiberty.parquet
  ✅ SAbDab + BALM: sabdab_embeddedby_balm.parquet
  ✅ SAbDab + ESM-2: sabdab_embeddedby_esm-2.parquet
  ✅ SAbDab + IgBERT: sabdab_embeddedby_igbert.parquet
  ✅ SAbDab + Parapred: sabdab_embeddedby_parapred.parquet
  ✅ DMS + AbLangPDB: dms_embeddedby_ablangpdb.parquet
  ✅ DMS + AbLangPre: dms_embeddedby_ablangpre.parquet
  ✅ DMS + AbLang2: dms_embeddedby_ablang2.parquet
  ✅ DMS + AbLang-Heavy: dms_embeddedby_ablang-heavy.parquet
  ✅ DMS + AntiBERTy: dms_embeddedby_antiberty.parquet
  ✅ DMS + BALM: dms_embeddedby_balm.parquet
  ✅ DMS + E

## Comprehensive Model Configuration

In [7]:
# Complete configuration for all model/dataset/metric combinations
CONFIGS = {
    # SAbDab Dataset Configurations
    "ablangpdb_sabdab_cosine": {
        "df_path": "ablangpdb_renameddatasets.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "AbLangPDB",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "ablangrbd_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_ablangrbd.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "AbLangRBD",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "ablangpre_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_ablangpre.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "AbLangPre",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "ablang2_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_ablang2.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "AbLang2",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "ablang_heavy_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_ablang-heavy.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "AbLang-Heavy",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "antiberty_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_antiberty.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "AntiBERTy",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "balm_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_balm.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "BALM",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "esm2_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_esm-2.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "ESM-2",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "igbert_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_igbert.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "IgBERT",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "parapred_sabdab_cosine": {
        "df_path": "sabdab_embeddedby_parapred.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "Parapred",
        "score_type": "cosine",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "seqid_sabdab": {
        "df_path": "ablangpdb_renameddatasets.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "SEQID",
        "score_type": "seq_identity",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    "cdrh3id_sabdab": {
        "df_path": "ablangpdb_renameddatasets.parquet",
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "CDRH3ID",
        "score_type": "cdrh3_identity",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab"
    },
    
    # DMS Dataset Configurations
    "ablangpdb_dms_cosine": {
        "df_path": "dms_embeddedby_ablangpdb.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "AbLangPDB",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "ablangrbd_dms_cosine": {
        "df_path": "ablangrbd_renameddatasets.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "AbLangRBD",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "ablangpre_dms_cosine": {
        "df_path": "dms_embeddedby_ablangpre.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "AbLangPre",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "ablang2_dms_cosine": {
        "df_path": "dms_embeddedby_ablang2.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "AbLang2",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "ablang_heavy_dms_cosine": {
        "df_path": "dms_embeddedby_ablang-heavy.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "AbLang-Heavy",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "antiberty_dms_cosine": {
        "df_path": "dms_embeddedby_antiberty.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "AntiBERTy",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "balm_dms_cosine": {
        "df_path": "dms_embeddedby_balm.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "BALM",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "esm2_dms_cosine": {
        "df_path": "dms_embeddedby_esm-2.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "ESM-2",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "igbert_dms_cosine": {
        "df_path": "dms_embeddedby_igbert.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "IgBERT",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "parapred_dms_cosine": {
        "df_path": "dms_embeddedby_parapred.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "Parapred",
        "score_type": "cosine",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "seqid_dms": {
        "df_path": "ablangrbd_renameddatasets.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "SEQID",
        "score_type": "seq_identity",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    "cdrh3id_dms": {
        "df_path": "ablangrbd_renameddatasets.parquet",
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "CDRH3ID",
        "score_type": "cdrh3_identity",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms"
    },
    
    # ABodyBuilder2 DTW CDRs Configurations
    "abodybuilder2_sabdab_dtw": {
        "df_path": "ablangpdb_renameddatasets.parquet",  # Not used for this score_type, but required for function signature
        "labels_val": "ablangpdb_train_val_label_mat.pt",
        "labels_test": "ablangpdb_train_test_label_mat.pt",
        "model_name": "ABodyBuilder2_DTW_CDRs",
        "score_type": "abodybuilder2_dtw_cdrs",
        "function": calculate_metrics.get_metrics,
        "dataset_type": "sabdab",
        "matrix_file_val": "sabdab_pairwise_cdr_sims_train_vs_val.npy",
        "matrix_file_test": "sabdab_pairwise_cdr_sims_train_vs_test.npy"
    },
    "abodybuilder2_dms_dtw": {
        "df_path": "ablangrbd_renameddatasets.parquet",  # Not used for this score_type, but required for function signature
        "labels_val": "dms_train_val_label_mat.pt",
        "labels_test": "dms_train_test_label_mat.pt",
        "model_name": "ABodyBuilder2_DTW_CDRs",
        "score_type": "abodybuilder2_dtw_cdrs",
        "function": calculate_metrics_dms.get_metrics_dms,
        "dataset_type": "dms",
        "matrix_file_val": "dms_pairwise_cdr_sims_train_vs_val.npy",
        "matrix_file_test": "dms_pairwise_cdr_sims_train_vs_test.npy"
    }
}

print(f"Configured {len(CONFIGS)} model/dataset/metric combinations:")
for name, config in CONFIGS.items():
    print(f"  • {name}: {config['model_name']} on {config['dataset_type']} using {config['score_type']}")

Configured 26 model/dataset/metric combinations:
  • ablangpdb_sabdab_cosine: AbLangPDB on sabdab using cosine
  • ablangrbd_sabdab_cosine: AbLangRBD on sabdab using cosine
  • ablangpre_sabdab_cosine: AbLangPre on sabdab using cosine
  • ablang2_sabdab_cosine: AbLang2 on sabdab using cosine
  • ablang_heavy_sabdab_cosine: AbLang-Heavy on sabdab using cosine
  • antiberty_sabdab_cosine: AntiBERTy on sabdab using cosine
  • balm_sabdab_cosine: BALM on sabdab using cosine
  • esm2_sabdab_cosine: ESM-2 on sabdab using cosine
  • igbert_sabdab_cosine: IgBERT on sabdab using cosine
  • parapred_sabdab_cosine: Parapred on sabdab using cosine
  • seqid_sabdab: SEQID on sabdab using seq_identity
  • cdrh3id_sabdab: CDRH3ID on sabdab using cdrh3_identity
  • ablangpdb_dms_cosine: AbLangPDB on dms using cosine
  • ablangrbd_dms_cosine: AbLangRBD on dms using cosine
  • ablangpre_dms_cosine: AbLangPre on dms using cosine
  • ablang2_dms_cosine: AbLang2 on dms using cosine
  • ablang_heavy_dms_cos

### Filter Available Configurations

In [8]:
# Check which configurations can actually run based on available files
available_configs = {}
missing_configs = []

for config_name, config in CONFIGS.items():
    files_to_check = [config["df_path"], config["labels_val"], config["labels_test"]]
    missing_files = [f for f in files_to_check if not os.path.exists(f)]
    
    if not missing_files:
        available_configs[config_name] = config
        print(f"✅ {config_name}: Ready to run")
    else:
        missing_configs.append(config_name)
        print(f"❌ {config_name}: Missing files - {missing_files}")

print(f"\n📊 Summary:")
print(f"  • Available configurations: {len(available_configs)}/{len(CONFIGS)}")
print(f"  • Missing configurations: {len(missing_configs)}")

if missing_configs:
    print(f"\n⚠️ Configurations that will be skipped: {', '.join(missing_configs)}")

if not available_configs:
    raise RuntimeError("❌ No configurations are available to run!")

✅ ablangpdb_sabdab_cosine: Ready to run
✅ ablangrbd_sabdab_cosine: Ready to run
✅ ablangpre_sabdab_cosine: Ready to run
✅ ablang2_sabdab_cosine: Ready to run
✅ ablang_heavy_sabdab_cosine: Ready to run
✅ antiberty_sabdab_cosine: Ready to run
✅ balm_sabdab_cosine: Ready to run
✅ esm2_sabdab_cosine: Ready to run
✅ igbert_sabdab_cosine: Ready to run
✅ parapred_sabdab_cosine: Ready to run
✅ seqid_sabdab: Ready to run
✅ cdrh3id_sabdab: Ready to run
✅ ablangpdb_dms_cosine: Ready to run
✅ ablangrbd_dms_cosine: Ready to run
✅ ablangpre_dms_cosine: Ready to run
✅ ablang2_dms_cosine: Ready to run
✅ ablang_heavy_dms_cosine: Ready to run
✅ antiberty_dms_cosine: Ready to run
✅ balm_dms_cosine: Ready to run
✅ esm2_dms_cosine: Ready to run
✅ igbert_dms_cosine: Ready to run
✅ parapred_dms_cosine: Ready to run
✅ seqid_dms: Ready to run
✅ cdrh3id_dms: Ready to run
✅ abodybuilder2_sabdab_dtw: Ready to run
✅ abodybuilder2_dms_dtw: Ready to run

📊 Summary:
  • Available configurations: 26/26
  • Missing con

## Optional: Pre-computed Thresholds

In [ ]:
# Optional: Pre-computed thresholds to skip threshold optimization
# Uncomment and set values to reuse previous results for faster execution

PRECOMPUTED_THRESHOLDS = {
    # Existing pre-computed thresholds (keep these values)
    "ablangpdb_sabdab_cosine": {
        "epitope_threshold": 0.5037,
        "antigen_threshold": 0.2697
    },
    "ablangrbd_sabdab_cosine": {
        "epitope_threshold": 0.7912,
        "antigen_threshold": -0.2969
    },
    "ablangpre_sabdab_cosine": {
        "epitope_threshold": 0.6941,
        "antigen_threshold": 0.5851
    },
    "parapred_sabdab_cosine": {
        "epitope_threshold": 0.9985,
        "antigen_threshold": 0.9974
    },
    "seqid_sabdab": {
        "epitope_threshold": 0.6684,
        "antigen_threshold": 0.3380
    },
    "cdrh3id_sabdab": {
        "epitope_threshold": 0.2727,
        "antigen_threshold": 0.0000
    },
    "ablangpdb_dms_cosine": {
        "epitope_threshold": -0.0419
    },
    "ablangrbd_dms_cosine": {
        "epitope_threshold": 0.8493
    },
    "ablangpre_dms_cosine": {
        "epitope_threshold": 0.6608
    },
    "parapred_dms_cosine": {
        "epitope_threshold": 0.9888
    },
    "seqid_dms": {
        "epitope_threshold": 0.6497
    },
    "cdrh3id_dms": {
        "epitope_threshold": 0.1905
    },

    "ablang2_sabdab_cosine": {
        "epitope_threshold": 0.9325,
        "antigen_threshold": 0.8962
    },
    "ablang_heavy_sabdab_cosine": {
        "epitope_threshold": 0.7814,
        "antigen_threshold": 0.6215
    },
    "antiberty_sabdab_cosine": {
        "epitope_threshold": 0.8272,
        "antigen_threshold": 0.6691
    },
    "balm_sabdab_cosine": {
        "epitope_threshold": 0.9420,
        "antigen_threshold": 0.9200
    },
    "esm2_sabdab_cosine": {
        "epitope_threshold": 0.9923,
        "antigen_threshold": 0.9866
    },
    "igbert_sabdab_cosine": {
        "epitope_threshold": 0.9721,
        "antigen_threshold": 0.9596
    },
    "ablang2_dms_cosine": {
        "epitope_threshold": 0.5205
    },
    "ablang_heavy_dms_cosine": {
        "epitope_threshold": 0.6617
    },
    "antiberty_dms_cosine": {
        "epitope_threshold": 0.6673
    },
    "balm_dms_cosine": {
        "epitope_threshold": 0.9378
    },
    "esm2_dms_cosine": {
        "epitope_threshold": .9930
    },
    "igbert_dms_cosine": {
        "epitope_threshold": 0.9314
    },
    
    "abodybuilder2_dms_dtw": {
        "epitope_threshold": 0.2496
    },
    "abodybuilder2_sabdab_dtw": {
        "epitope_threshold": 0.0148,
        "antigen_threshold": 0.0000
    },
}

use_precomputed = len(PRECOMPUTED_THRESHOLDS) > 0
if use_precomputed:
    print(f"🔄 Using precomputed thresholds for {len(PRECOMPUTED_THRESHOLDS)} configurations")
    for config_name, thresholds in PRECOMPUTED_THRESHOLDS.items():
        print(f"  • {config_name}: {thresholds}")
else:
    print("🆕 Computing fresh thresholds for all configurations")

🔄 Using precomputed thresholds for 24 configurations
  • ablangpdb_sabdab_cosine: {'epitope_threshold': 0.5037, 'antigen_threshold': 0.2697}
  • ablangrbd_sabdab_cosine: {'epitope_threshold': 0.7912, 'antigen_threshold': -0.2969}
  • ablangpre_sabdab_cosine: {'epitope_threshold': 0.6941, 'antigen_threshold': 0.5851}
  • parapred_sabdab_cosine: {'epitope_threshold': 0.9985, 'antigen_threshold': 0.9974}
  • seqid_sabdab: {'epitope_threshold': 0.6684, 'antigen_threshold': 0.338}
  • cdrh3id_sabdab: {'epitope_threshold': 0.2727, 'antigen_threshold': 0.0}
  • ablangpdb_dms_cosine: {'epitope_threshold': -0.0419}
  • ablangrbd_dms_cosine: {'epitope_threshold': 0.8493}
  • ablangpre_dms_cosine: {'epitope_threshold': 0.6608}
  • parapred_dms_cosine: {'epitope_threshold': 0.9888}
  • seqid_dms: {'epitope_threshold': 0.6497}
  • cdrh3id_dms: {'epitope_threshold': 0.1905}
  • ablang2_sabdab_cosine: {'epitope_threshold': 0.9325, 'antigen_threshold': 0.8962}
  • ablang_heavy_sabdab_cosine: {'epitope

## Run Comprehensive Benchmarks

In [10]:
# Execute all available configurations
execution_results = {}
failed_configs = []

print(f"\n{'='*70}")
print(f"COMPREHENSIVE BENCHMARK EXECUTION")
print(f"{'='*70}")
print(f"Total configurations to run: {len(available_configs)}")
print(f"Recalculate summary metrics: {RECALCULATE_SUMMARYMETRICS}")

for i, (config_name, config) in enumerate(available_configs.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{i}/{len(available_configs)}] Running: {config_name}")
    print(f"Model: {config['model_name']}, Dataset: {config['dataset_type']}, Score: {config['score_type']}")
    print(f"{'='*70}")
    
    # Check if summary files already exist and flag is False
    if not RECALCULATE_SUMMARYMETRICS:
        summary_files = get_summary_file_paths(config['model_name'], config['dataset_type'], OUTPUT_FOLDER)
        all_summaries_exist = all(os.path.exists(f) for f in summary_files)
        
        if all_summaries_exist:
            print(f"📋 Summary files already exist, skipping recalculation...")
            read_and_display_summary_results(summary_files, config['model_name'], config['dataset_type'])
            execution_results[config_name] = "✅ Loaded from existing files"
            print(f"\n✅ [{i}/{len(available_configs)}] {config_name} loaded from existing files!")
            continue
        else:
            print(f"📋 Some summary files missing, proceeding with calculation...")
    
    try:
        # Prepare arguments
        args = {
            "df_path": config["df_path"],
            "labels_file_val": config["labels_val"],
            "labels_file_test": config["labels_test"],
            "score_type": config["score_type"],
            "model_name": config["model_name"],
            "output_folder": OUTPUT_FOLDER
        }
        
        # Add matrix file paths for ABodyBuilder2 DTW configurations
        if config["score_type"] == "abodybuilder2_dtw_cdrs":
            args["matrix_file_val"] = config["matrix_file_val"]
            args["matrix_file_test"] = config["matrix_file_test"]
        
        # Add precomputed thresholds if available
        if config_name in PRECOMPUTED_THRESHOLDS:
            thresholds = PRECOMPUTED_THRESHOLDS[config_name]
            if config["dataset_type"] == "sabdab":
                if "epitope_threshold" in thresholds:
                    args["epitope_threshold"] = thresholds["epitope_threshold"]
                if "antigen_threshold" in thresholds:
                    args["antigen_threshold"] = thresholds["antigen_threshold"]
            elif config["dataset_type"] == "dms":
                if "epitope_threshold" in thresholds:
                    args["epitope_threshold"] = thresholds["epitope_threshold"]
        
        # Execute the benchmark
        config["function"](**args)
        
        execution_results[config_name] = "✅ Success"
        print(f"\n✅ [{i}/{len(available_configs)}] {config_name} completed successfully!")
        
    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        execution_results[config_name] = error_msg
        failed_configs.append(config_name)
        print(f"\n❌ [{i}/{len(available_configs)}] {config_name} failed: {str(e)}")
        continue

print(f"\n{'='*70}")
print("COMPREHENSIVE BENCHMARK EXECUTION SUMMARY")
print(f"{'='*70}")

for config_name, result in execution_results.items():
    print(f"{config_name:30} {result}")

successful_configs = len(available_configs) - len(failed_configs)
print(f"\n📊 Results:")
print(f"  • Successful: {successful_configs}/{len(available_configs)}")
print(f"  • Failed: {len(failed_configs)}")

if failed_configs:
    print(f"\n⚠️ Failed configurations: {', '.join(failed_configs)}")
else:
    print("\n🎉 All available configurations completed successfully!")


COMPREHENSIVE BENCHMARK EXECUTION
Total configurations to run: 26
Recalculate summary metrics: False

[1/26] Running: ablangpdb_sabdab_cosine
Model: AbLangPDB, Dataset: sabdab, Score: cosine
📋 Summary files already exist, skipping recalculation...

📊 Loading existing results for AbLangPDB on sabdab:

--- Epitope-level performance ---
Model: AbLangPDB
Dataset: sabdab_ep
Score_Type: cosine
ROC_AUC: 0.809048
Average_Precision: 0.541921
F1_Score: 0.556708
Threshold: 0.503700

--- Antigen-level performance ---
Model: AbLangPDB
Dataset: sabdab_ag
Score_Type: cosine
ROC_AUC: 0.788735
Average_Precision: 0.508352
F1_Score: 0.504352
Threshold: 0.269700

✅ [1/26] ablangpdb_sabdab_cosine loaded from existing files!

[2/26] Running: ablangrbd_sabdab_cosine
Model: AbLangRBD, Dataset: sabdab, Score: cosine
📋 Summary files already exist, skipping recalculation...

📊 Loading existing results for AbLangRBD on sabdab:

--- Epitope-level performance ---
Model: AbLangRBD
Dataset: sabdab_ep
Score_Type: cos

## Generate Comprehensive Excel Report

In [11]:
# Generate summary statistics
print("\n📊 Generating summary statistics...")
print_summary_stats(OUTPUT_FOLDER)

print("\n" + "="*70)
print("GENERATING COMPREHENSIVE EXCEL REPORT")
print("="*70)

try:
    # Generate the Excel file
    excel_path = generate_results_excel(
        output_folder=OUTPUT_FOLDER,
        excel_filename=EXCEL_FILENAME
    )
    
    print(f"\n🎉 Comprehensive Excel report generated successfully!")
    print(f"📁 File location: {excel_path}")
    print(f"📏 File size: {os.path.getsize(excel_path):,} bytes")
    
    # Provide usage instructions
    print(f"\n📖 Excel Report Contents:")
    print(f"  • Models as rows (AbLangPDB, AbLangRBD, AbLangPre, SEQID, CDRH3ID)")
    print(f"  • Datasets grouped as column headers (SAbDab, DMS)")
    print(f"  • Metrics: ROC-AUC, Average Precision, F1 Score")
    print(f"  • Best performance: Bold formatting")
    print(f"  • Second best: Italic formatting")
    print(f"  • Values rounded to 4 decimal places")
    
except Exception as e:
    print(f"❌ Error generating Excel report: {str(e)}")
    print("\nDebugging information:")
    print(f"  • Output folder: {OUTPUT_FOLDER}")
    print(f"  • Files in folder: {len(os.listdir(OUTPUT_FOLDER))}")
    
    # List summary files found
    import glob
    summary_files = glob.glob(os.path.join(OUTPUT_FOLDER, "*summarymetrics.txt"))
    print(f"  • Summary files found: {len(summary_files)}")
    for f in summary_files[:5]:  # Show first 5
        print(f"    - {os.path.basename(f)}")
    if len(summary_files) > 5:
        print(f"    - ... and {len(summary_files)-5} more")


📊 Generating summary statistics...

=== Summary Statistics ===
Total summary files found: 39
Unique models: 13 (ABodyBuilder2_DTW_CDRs, AbLang-Heavy, AbLang2, AbLangPDB, AbLangPre, AbLangRBD, AntiBERTy, BALM, CDRH3ID, ESM-2, IgBERT, Parapred, SEQID)
Unique datasets: 3 (dms, sabdab_ag, sabdab_ep)
Unique score types: 4 (abodybuilder2_dtw_cdrs, cdrh3_identity, cosine, seq_identity)
Best ROC_AUC: AbLangRBD on dms (0.8442)
Best Average_Precision: AbLangRBD on dms (0.6401)
Best F1_Score: AbLangRBD on dms (0.5926)

GENERATING COMPREHENSIVE EXCEL REPORT
Found 39 summary files
Creating pivot table...
Pivot table created with 13 models and 9 metric columns
Ranking values for formatting...
Exporting to Excel: output_csvs/comprehensive_benchmarking_results.xlsx
✅ Excel file generated successfully: output_csvs/comprehensive_benchmarking_results.xlsx

🎉 Comprehensive Excel report generated successfully!
📁 File location: output_csvs/comprehensive_benchmarking_results.xlsx
📏 File size: 5,977 bytes

📖

## Final Summary and Analysis

In [12]:
print("\n" + "="*70)
print("COMPREHENSIVE PIPELINE COMPLETION SUMMARY")
print("="*70)

print(f"\n🔧 Configuration:")
print(f"  • Recalculated embeddings: {RECALCULATE_EMBEDDINGS}")
print(f"  • Used precomputed thresholds: {use_precomputed}")
print(f"  • Batch size: {BATCH_SIZE}")

print(f"\n📈 Benchmarking Results:")
print(f"  • Total configurations possible: {len(CONFIGS)}")
print(f"  • Configurations attempted: {len(available_configs)}")
print(f"  • Successful runs: {successful_configs}")
print(f"  • Failed runs: {len(failed_configs)}")

if os.path.exists(os.path.join(OUTPUT_FOLDER, EXCEL_FILENAME)):
    print(f"\n📊 Excel Report:")
    print(f"  • Status: ✅ Generated successfully")
    print(f"  • Location: {os.path.join(OUTPUT_FOLDER, EXCEL_FILENAME)}")
    print(f"  • Ready for analysis and sharing")
else:
    print(f"\n📊 Excel Report:")
    print(f"  • Status: ❌ Generation failed")
    print(f"  • Check error messages above")

print(f"\n🔬 Models Configured for Benchmarking:")
all_models = set()
for config_name, config in CONFIGS.items():
    all_models.add(f"{config['model_name']} ({config['score_type']})")
        
for model in sorted(all_models):
    print(f"  • {model}")

print(f"\n📊 Datasets Configured:")
datasets_configured = set()
for config_name, config in CONFIGS.items():
    datasets_configured.add(config['dataset_type'].upper())
        
for dataset in sorted(datasets_configured):
    print(f"  • {dataset}")

print(f"\n🎯 Next Steps:")
print(f"  1. 📊 Open the Excel report for comprehensive performance comparison")
print(f"  2. 🔍 Identify best-performing models for each dataset")
print(f"  3. 📈 Analyze performance patterns across different similarity metrics")
print(f"  4. 📋 Share results with your research team")
print(f"  5. 📝 Consider additional analyses based on findings")

if failed_configs:
    print(f"\n⚠️ Failed Configurations to Investigate:")
    for config in failed_configs:
        print(f"  • {config}: {execution_results[config]}")

if missing_configs:
    print(f"\n❓ Configurations Not Attempted (Missing Files):")
    for config in missing_configs:
        print(f"  • {config}")

print(f"\n💡 Model Coverage Summary:")
print(f"  • Total unique models: {len(all_models)}")
print(f"  • Embedding-based models: AbLangPDB, AbLangRBD, AbLangPre, AbLang2,")
print(f"    AbLang-Heavy, AntiBERTy, BALM, ESM-2, IgBERT, Parapred")
print(f"  • Sequence-based models: SEQID, CDRH3ID")
print(f"  • Total configurations: {len(CONFIGS)}")

print(f"\n🏁 Comprehensive benchmarking pipeline completed!")
print(f"\n📄 Report: {os.path.join(OUTPUT_FOLDER, EXCEL_FILENAME)}")


COMPREHENSIVE PIPELINE COMPLETION SUMMARY

🔧 Configuration:
  • Recalculated embeddings: False
  • Used precomputed thresholds: True
  • Batch size: 256

📈 Benchmarking Results:
  • Total configurations possible: 26
  • Configurations attempted: 26
  • Successful runs: 26
  • Failed runs: 0

📊 Excel Report:
  • Status: ✅ Generated successfully
  • Location: output_csvs/comprehensive_benchmarking_results.xlsx
  • Ready for analysis and sharing

🔬 Models Configured for Benchmarking:
  • ABodyBuilder2_DTW_CDRs (abodybuilder2_dtw_cdrs)
  • AbLang-Heavy (cosine)
  • AbLang2 (cosine)
  • AbLangPDB (cosine)
  • AbLangPre (cosine)
  • AbLangRBD (cosine)
  • AntiBERTy (cosine)
  • BALM (cosine)
  • CDRH3ID (cdrh3_identity)
  • ESM-2 (cosine)
  • IgBERT (cosine)
  • Parapred (cosine)
  • SEQID (seq_identity)

📊 Datasets Configured:
  • DMS
  • SABDAB

🎯 Next Steps:
  1. 📊 Open the Excel report for comprehensive performance comparison
  2. 🔍 Identify best-performing models for each dataset
  3. 